# NVIDIA Displacement Micromap Python Toolkit

This notebook reviews the Micromap Python Toolkit.  It first reviews what Micromap data is, summarizes its use-cases in rendering, and gives a hands on introduction to the module.  The tutorial loads mesh data stored in the Universal Scene Description (USD) format, converts it to Numpy arrays and Python objects suitable for the Python module, and demonstrates how to use the toolkit to generate compressed displacement micromap data given high and low resolution meshes, including an optional heightmap.  It also demonstrates how to resample the input textures suitable to the UV-space of the displaced geometry.

<center><img src="crab.jpg"></img></center>

### Prerequisites

This notebook was developed and tested using USD 22.11, however it uses only minimal USD so you may be able to use similar versions.  You will need to install a version of Pixar USD by running the cell below, or you can download your own and add it to the Python path for the environment you're running this Jupyter notebook from.  Some pre-built binaries are made available here (https://developer.nvidia.com/usd#bin).  numpy and numba are also required for performant tweaking of attributes, and py7zr for extracting the downloaded notebook assets.  The following cell can uninstall these libraries if/when necessary.  

In [ ]:
import sys
!{sys.executable} -m pip install usd-core==22.11 numpy numba py7zr

In [ ]:
#!{sys.executable} -m pip uninstall usd-core numpy numba py7zr

Download and extract the notebook assets:

In [ ]:
from utils.assets import fetch_and_extract

remote_file = 'https://developer.download.nvidia.com/ProGraphics/nvpro-samples/notebook_assets.7z'

fetch_and_extract(remote_file, '.', './assets', force=False)

## Micro-Mesh Overview

Before jumping into the Displacement Micromap Python Toolkit, lets briefly review what micro-mesh technology is and how we can create it from our assets.  For a more detailed description, refer to (https://github.com/NVIDIAGameWorks/Displacement-MicroMap-SDK).  Micro-meshes with micromap data are an optimized and compressed set of geometry and at-runtime displacement and allow for reduced framebuffer requirements for high-resolution, displaced meshes.  Micro-meshes also enable faster bounding volume heirarchy (BVH) build and traversal times.  It comes with a trade-off in that while the BVH is quicker to build and traverse and requires less memory than a standard BVH and high-resolution mesh, there is a performance overhead when ray tracing as each ray trace into the leaf will incur a penalty to decode and intersect.  This will be discussed a bit more after describing the data.

In a nutshell, displaced micromap data is a compressed representation of high-resolution geometric details coupled with a low-resolution triangle mesh -- similar to a texture but instead of sampled in UV-space the displacements are uniformly sampled on a barycentric grid within each base triangle.  This data is created by <i>baking</i> a high-resolution mesh or tesselated heightmap surface onto the low-resolution mesh and storing displacement values at <i>interpolated positions</i> within each triangle of the low-resolution mesh.  During the baking process, rays are traced from these interpolated positions to their intersection with the high-resolution mesh.  This distance determines their <i>displacement value</i> and when displaced the <i>micro-vertex position</i>.  Per-vertex <i>direction vectors</i> are interpolated across the low-resolution triangle using barycentric coordinates to become the <i>interpolated displacement vector</i>. These direction vectors could simply be vertex normals, or otherwise non-normalized custom vectors supplied by the user.  In the end, these direction vectors form a hull of varying thickness around the base mesh, and it is within this hull that all displacements occur. As such, this hull can be used as a quick measure of rendering efficiency -- the tighter the hull fits to the mesh, the faster potential displaced microtriangles can be ignored by the ray tracing hardware. In addition, a tighter hull allows for more resolution in the compressed displacements.

Micro-vertex attribute interpolation, e.g. for positions or direction vectors, relies on the barycentric coordinates to place the micro-mesh displacements evenly across these base triangles, so as you may have guessed, isotropic triangles are much preferred in order to maximize resources. Triangles that are flat or elongated are still functional, they just overdefine the displacements and are not preferred.  As such, care should be taken when authoring the high-resolution meshes to avoid them.  Or, when applicable, a remeshing algorithm that can retriangluate the mesh to isotropic triangles that uniformly represent the displacement frequency in the high-resolution mesh can be used to generate the low-resolution mesh.

<center><img src="microdisplacement.png"></img></center>

<a id="toc"></a>
## Displacement Micromap Python Toolkit

The Python Toolkit presented here is a higher-level encapsulation of the Displacement Micromap SDK and related toolkits: 
* [Displacement-MicroMap-SDK](https://github.com/NVIDIAGameWorks/Displacement-MicroMap-SDK)
   * Low-level C-style API for creating, compressing, and optimizing micromap data
   * Built upon Vulkan SDK for GPU accelerated ray tracing used when generating displacements
   * Intended for tight integration into applications and tools utilizing C/C++
* [Displacement-MicroMap-Toolkit](https://github.com/NVIDIAGameWorks/Displacement-MicroMap-Toolkit)
   * Higher-level suite of applications for creating, compressing, and optimizing micromap data from .gltf-formatted data
   * Includes support for remeshing and pre-tessellation of meshes to support more efficient micromeshes
   * Can be integrated into asset pipelines that support .gltf 
* [Displacement-MicroMap-BaryFile](https://github.com/NVIDIAGameWorks/Displacement-MicroMap-BaryFile)
   * .bary file format and container used to store micromesh-related attributes to disk
   * .gltf extension that adds support for .bary to .gltf files
* [NVIDIA Micro-Mesh Technology Landing Page](https://developer.nvidia.com/rtx/ray-tracing/micro-mesh)
   * More information and links related to displaced micro-mesh and opacity micro-maps

The Python Toolkit is intended an an entry point into the NVIDIA Displacement Micromap SDK for integration into DCC apps and in-house tools that leverage Python.  It exposes similar features as the Displacement Micromap Toolkit does for command line / pipelined assets but instead inside a Python environment.

The Displacement Micromap Python Toolkit currently supports a number of workflows:

1) [Baking Micro-Mesh Data](#baking_micromesh_data)
   * [Settings](#baker_settings)
   * [Texture Resampling](#texture_resampling)
   * [Heightmaps](#heightmaps)
   * [Baking](#baking)
   * [Saving to .bary File Format](#save_to_bary)
   * [Restoring from .bary File Format](#restore_from_bary)
   * [Saving to USD](#save_to_usd)
2) [Displaced Mesh Generation](#displaced_mesh)
3) [Remeshing](#remeshing)
4) [Pre-tessellating](#pretessellation)

The rest of the notebook will introduce each workflow and how the Displacement Micromap Python Toolkit can be used to accomplish them.

## Baking Micro-Mesh Data<a id="baking_micromesh_data"></a>

Baking always requires a low-resolution or target mesh on which to place the displacements.  For the source surface, you can either use a high-resolution mesh, a low-resolution mesh with a heightmap, or both a high-resolution mesh and a heightmap.  In case with a heightmap, it will be applied to the mesh during the baking process in order to construct the final high-resolution surface.

Before we can create any displacements we need to load the source data.  The next cell loads the given .usd stage into memory and briefly examines its contents.

In [ ]:
from utils import mesh
from pxr import Usd, UsdGeom

stage = Usd.Stage.Open('./assets/simplewall/simplewall.usd')

for prim in stage.Traverse():
    if prim.IsA(UsdGeom.Mesh):
        mesh_prim = UsdGeom.Mesh(prim)
        print(f'{mesh_prim.GetPrim().GetPrimPath()}: vertices={len(mesh_prim.GetPointsAttr().Get())}, faces={int(len(mesh_prim.GetFaceVertexIndicesAttr().Get())/3)}')
        
low_prim = stage.GetPrimAtPath('/World/Low/Simplewall')
high_prim = stage.GetPrimAtPath('/World/High/Simplewall')

# Get the low and high resolution primitives from the stage and convert to the necessary format:
low_mesh, low_mesh_xform = mesh.get_mesh(low_prim)
high_mesh, high_mesh_xform = mesh.get_mesh(high_prim)

Here we just find some texture filepaths we'll use later in the notebook:

In [ ]:
import os
import pathlib

textures_folder = './assets/simplewall'
resampled_resolution = 512

# Find all the .png textures to resample
filenames = [filename for filename in os.listdir(textures_folder) if 'png' in filename]

# Create the output folder, if necessary
resampled_textures_folder = './assets/simplewall/resampled'
if not os.path.isdir(resampled_textures_folder):
    os.mkdir(resampled_textures_folder)

heightmap_filenames = [filename for filename in os.listdir(textures_folder) if 'height' in filename]
heightmap_filename = str(pathlib.Path(textures_folder) / heightmap_filenames[0]) if len(heightmap_filenames) > 0 else ""
print(f'Heightmap filename {heightmap_filename}')

A context object is required for all toolkit operations.  This context allocates internal resources that can be reused across multiple invocations of the toolkit operations so it's handy to keep and reuse as needed.

In [ ]:
import micromesh_python as pymm

context = pymm.createContext(verbosity=pymm.Verbosity.Info)

### Settings<a id="baker_settings"></a>

Now lets create the settings object which will house some tweakable parameters that effect the resulting displaced micromap data:

In [ ]:
settings = pymm.BakerSettings()

# Control amount of subdivision levels from 0-5
settings.level = 5

# True to allow the baker to automatically fit the direction bounds to the localized surface
# False to use global min/max bounds fitting
settings.fitDirectionBounds = True

# True to enable compression, false otherwise
# Note: the graphics driver cannot render uncompressed micromesh data
settings.enableCompression = True

# Compression factor in range 0-100
settings.minPSNR = 45.0

# Subdivision method:
#   Uniform to use constant subdivision level (above) for all base triangles
#   Adaptive3D to select base triangle subdivision level using details in local geometry
#   AdaptiveUV to select base triangle subdivision level using details in UV+heightmap
settings.subdivMethod = pymm.SubdivMethod.Adaptive3D

### Texture Resampling<a id="texture_resampling"></a>

If you have assets with textures, it is important to resample those textures so they can be rendered correctly in the displaced mesh.  We can build construct a list of textures to resample for the baker utility.  In addition we can find the heightmap.

In [ ]:
resampler_inputs = []
for filename in filenames:
    resampler_input = pymm.ResamplerInput()
    resampler_input.input.filepath = str(pathlib.Path(textures_folder) / filename)
    resampler_input.input.type = pymm.TextureType.Generic
    resampler_input.output.filepath = str(pathlib.Path(resampled_textures_folder) / filename)
    resampler_input.output.width = resampled_resolution
    resampler_input.output.height = resampled_resolution
    resampler_input.output.format = pymm.TextureFormat.RGBA8Unorm
    resampler_inputs.append(resampler_input)
    print(f'Resampling {resampler_input.input.filepath} to {resampler_input.output.filepath}')

### Baking<a id="baking"></a>

With input meshes loaded, settings initialized, and resampled textures listed, we can set up the baker input.  There are a few ways we can bake displaced micromap data:

* Low-resolution mesh + heightmap
* Low-resolution mesh + high-resolution mesh
* Low-resolution mesh + heightmap + high-resolution mesh

All three options require the low-resolution mesh but the baker will also work with a combination of heightmap and high-resolution mesh inputs.  If both heightmap and high-resolution mesh inputs are supplied, then the high-resolution mesh is displaced using the heighmap and that displaced mesh is used as the tracing surface.

In [ ]:
baker_input = pymm.BakerInput()
baker_input.settings = settings
baker_input.baseMesh = low_mesh
baker_input.baseMeshTransform = low_mesh_xform
baker_input.referenceMesh = high_mesh
baker_input.referenceMeshTransform = high_mesh_xform
baker_input.heightmap.filepath = heightmap_filename
baker_input.heightmap.scale = 0.133119
baker_input.heightmap.bias = -0.074694
if len(resampler_inputs) > 0:
    baker_input.resamplerInput = resampler_inputs

Now we can call the baker:

In [ ]:
micromesh_data = pymm.bakeMicromesh(context, baker_input)

### Saving to .bary File Format<a id="save_to_bary"></a>

Alternatively, we can save out to a micromesh-specific .bary format.  This format will store all of the necessary micromap data and is fully supported by the C/C++ Displacement Micromnap SDK.  The Python Toolkit allows you to save to simply write the micromesh to .bary easily:

In [ ]:
bary_file = './assets/low.bary'
pymm.writeBary(context, bary_file, low_mesh, micromesh_data, forceOverwrite=True)

### Restoring from .bary File Format<a id="restore_from_bary"></a>

Conversely, we can load a .bary file similarly:

In [ ]:
restored_micromesh_data = pymm.readBary(context, bary_file, low_mesh)

import numpy as np
print(np.array_equal(restored_micromesh_data.values, micromesh_data.values))

In [ ]:
displaced_mesh = pymm.displaceMicromesh(context, low_mesh, micromesh_data)
print(f'Triangles in displaced mesh: {displaced_mesh.triangleVertices.shape[0]}')

Now we can convert that displaced mesh back to USD and save out the stage:

In [ ]:
temp_stage = Usd.Stage.CreateInMemory()
mesh.create_mesh('/World/displaced/simplewall', displaced_mesh, temp_stage)
temp_stage.Export('./assets/displaced.usd')

### Saving to USD<a id="save_to_usd"></a>

The resulting displaced micromap data can also be stored back into USD as primvar attributes and utilized by a renderer supporting micro-meshes.  Here is a utility function that will create the necessary primvars and convert to USD from the numpy arrays in the micromesh data object:

In [ ]:
mesh.store_micromesh_primvars(micromesh_data, low_prim)

## Remeshing<a id="remeshing"></a>

If the only mesh available is a high-resolution mesh we can utilize a remeshing algorithm to produce a suitable low-resolution base mesh. Or if a low-resolution mesh exists but has many non-isotropic or long and thin triangles, we can rework that mesh to be more suitable for micro-meshes.  The Displacement Micromap Toolkit provides a GPU-accelerated remeshing algorithm that does just that.  It is controllable through a number of settings to help guide the algorithm using error thresholds, decimation amounts, and even a guide on upper bound for triangle count.

Lets load a new, more detailed mesh and show how to remesh it using the Toolkit:

In [ ]:
from pxr import Usd, UsdGeom

reefcrab_stage = Usd.Stage.Open('./assets/reefcrab/reefcrab.usd')
        
reefcrab_prim = reefcrab_stage.GetPrimAtPath('/World/High/Reefcrab')

reefcrab_mesh, reefcrab_mesh_xform = mesh.get_mesh(reefcrab_prim)
print(f'Triangles in low-resolution mesh: {reefcrab_mesh.triangleVertices.shape[0]}')

In [ ]:
remesher_settings = pymm.RemesherSettings()
remesher_settings.decimationRatio=0.2
remesher_settings.errorThreshold=90
remesher_settings.maxSubdivLevel=4

remeshed_mesh = pymm.remesh(context, reefcrab_mesh, remesher_settings)

print(f'Triangles in remeshed mesh: {remeshed_mesh.triangleVertices.shape[0]}')
print(f'Actual decimation ratio: {remeshed_mesh.triangleVertices.shape[0] / reefcrab_mesh.triangleVertices.shape[0]:.2f}')

### Pre-Tessellation<a id="pretessellation"></a>

It might be necessary to introduce <i>more</i> detail into an mesh in order to produce a desireable micro-mesh. For example, consider a triangle mesh containing two triangles defining a simple heightmapped surface.  There may be more detail in that heightmap that cannot be managed by even the highest level of displaced micro-mesh subdivision level.  We can accomplish this by running a pre-tessellation algorithm. 

In [ ]:
pretessellator_settings = pymm.PreTessellatorSettings()
pretessellator_settings.maxSubdivLevel = 5
pretessellator_settings.edgeLengthBased = True

pretessellated_mesh = pymm.preTessellate(context, low_mesh, pretessellator_settings)
print(f'Triangles in low-resolution mesh: {low_mesh.triangleVertices.shape[0]}')
print(f'Triangles in remeshed mesh: {pretessellated_mesh.triangleVertices.shape[0]}')

### Full Pipeline<a id="full_pipeline"></a>

Now lets throw it all together and bake out displacement micromap data for the simple wall asset.  Since the low-resolution mesh has only 2 triangles and the displacement heightmap has a resolution of 2048, in order to reconstruct the full displaced heightmap as a micro-mesh, we need to pre-tesselate the base mesh to support the higher level of detail required for producing a micro-mesh from the heightmap:

In [ ]:
pretessellator_settings = pymm.PreTessellatorSettings()
pretessellator_settings.maxSubdivLevel = 5
pretessellator_settings.edgeLengthBased = True

print("Pre-tessellating...")
pretessellated_mesh = pymm.preTessellate(context, low_mesh, pretessellator_settings)
print(f'Triangles in low-resolution mesh: {low_mesh.triangleVertices.shape[0]}')
print(f'Triangles in pre-tessellated mesh: {pretessellated_mesh.triangleVertices.shape[0]}')

In [ ]:
settings = pymm.BakerSettings()
settings.level = 5
settings.fitDirectionBounds = True
settings.enableCompression = True
settings.minPSNR = 45.0
settings.subdivMethod = pymm.SubdivMethod.Uniform

baker_input = pymm.BakerInput()
baker_input.settings = settings
baker_input.baseMesh = pretessellated_mesh
baker_input.baseMeshTransform = low_mesh_xform
baker_input.referenceMesh = pretessellated_mesh
baker_input.referenceMeshTransform = low_mesh_xform
baker_input.heightmap.filepath = heightmap_filename
baker_input.heightmap.scale = 0.133119
baker_input.heightmap.bias = -0.074694

print("Baking...")
micromesh_data = pymm.bakeMicromesh(context, baker_input)

print("Displacing...")
displaced_mesh = pymm.displaceMicromesh(context, pretessellated_mesh, micromesh_data)

print(f'Triangles in displaced mesh: {displaced_mesh.triangleVertices.shape[0]}')

Clean up the context when done:

In [ ]:
context = None